## Import Libraries

In [1]:
import pandas as pd
import re
import spacy

import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE

from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
import keras
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


Using TensorFlow backend.


In [ ]:
pip install tensorflow


In [ ]:
# pip install keras


In [2]:
df = pd.read_csv('lyrics.csv', index_col='index')


In [ ]:
#let's look at genre value counts

df.genre.value_counts()

In [3]:
#filter DF to four genres
genre_list = ['Rock', 'Pop', 'Country', 'Hip-Hop']


df1 = df.loc[(df['genre'] == 'Rock') | (df['genre'] == 'Pop') | (df['genre'] == 'Country') | (df['genre'] == 'Hip-Hop')]

In [ ]:
#check the shape of the new dataframe
df1.shape

In [ ]:
df1.isna().sum()

In [4]:
#drop missing values from df
df1.dropna(inplace=True)


/Users/flatironschool/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df1.shape

## Cleaning Database

In [ ]:
song = df1.lyrics[1]
song

In [5]:
#create a function that clean and tokenizes lyrics

def clean_tokenize_lyrics(song):
    word_list = []
    tokenized_lyrics = word_tokenize(song)
     #remove all tokens that are not alphabetic
    words = [word for word in tokenized_lyrics if word.isalpha()]
    for word in words:
        lower_word = word.lower()
        word_list.append(lower_word)
    return word_list

In [ ]:
# clean_tokenize_lyrics(song)

In [6]:
#clean lyrics for all songs
df1['lyrics'] = df1.lyrics.apply(lambda x: clean_tokenize_lyrics(x))


/Users/flatironschool/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Stemm Clean Lyrics

In [18]:
stemmer = SnowballStemmer('english')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 

In [8]:
df1['cleaned_lyrics_stemmed'] = df1['lyrics'].apply(lambda x: [stemmer.stem(word) 
                                                for word in x if word not in stop_words])

/Users/flatironschool/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Lemmantize Clean Lyrics

In [9]:
df1['cleaned_lyrics_lemm'] = df1['lyrics'].apply(lambda x: [lemmatizer.lemmatize(word) 
                                            for word in x if word not in stop_words])

/Users/flatironschool/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
#remap genres to numerical values
genre_dict = {'Rock':1, 'Pop':2, 'Hip-Hop': 3, 'Country':4}

df1['genre'] = df1['genre'].replace(genre_dict)

/Users/flatironschool/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# df1['genre'].value_counts()

### StopWords

In [7]:
#let's engineer a new feature for proportion of stopwords for each song


stopword_pro = []
for song in df1['lyrics']:
    list_stop_words = [w for w in song if w in stop_words]
    num_stopwords = len(list_stop_words)
    total_words = len(song)
    stopword_pro.append(num_stopwords/total_words)



In [8]:
#multiply each value by 100 to get percentage
stopword_pro_percentage = np.array(stopword_pro ) *100  

In [9]:
#add to feature to dataframe
df1['stopword_pro_percentage'] = stopword_pro_percentage

In [10]:
df1.groupby('genre')['stopword_pro_percentage'].mean().reset_index()

,genre,stopword_pro_percentage
0,1,23.266378
1,2,23.143565
2,3,23.302156
3,4,23.051414


 
 Not much of a difference between the number of stop words 
    between all 4 genres. With this insight we can eliminate 
    those word tokens from the lyrcs

In [19]:
#let's remove stopwords from the lyrics
all_filtered_lyrics = []
for song in df1['lyrics']:
    filtered_lyrics = [w for w in song if not w in stop_words] 
    all_filtered_lyrics.append(filtered_lyrics)


In [20]:
df1['filtered_lyrics'] = all_filtered_lyrics

1872

### Save Clean Database as csv file

In [23]:
df1.to_csv('clean_lyric_df.csv')
# df1 = pd.read_csv('clean_lyric_df.csv', index_col=0)

## NLP Preprocessing

### Vectorize Words

In [ ]:
#create function to vectorize song

def count_vectorize(song, vocab=None):
    if vocab:
        unique_words = vocab
    else:
        unique_words = list(set(song))
    
    song_dict = {i:0 for i in unique_words}
    
    for word in song:
        song_dict[word] += 1
    
    return song_dict

In [ ]:
BoW = count_vectorize(df1['lyrics'][0])

### Term Frequency

In [ ]:
#create term frequency function
def term_frequency(BoW_dict):
    total_word_count = sum(BoW_dict.values())
    
    for ind, val in BoW_dict.items():
        BoW_dict[ind] = val/ total_word_count
    
    return BoW_dict

In [ ]:
#create a list of dictionaries
def create_list_of_BoW(song_lyrics):

    list_of_dictionaries = []

    for song in song_lyrics:
        BoW = count_vectorize(song)

        list_of_dictionaries.append(BoW)
    return list_of_dictionaries

list_of_dictionaries = create_list_of_BoW(df1['lyrics'])

### Inverse Document Frequency

In [ ]:
def inverse_document_frequency(list_of_dicts):
    vocab_set = set()
    # Iterate through list of dfs and add index to vocab_set
    for d in list_of_dicts:
        for word in d.keys():
            vocab_set.add(word)
    
    # Once vocab set is complete, create an empty dictionary with a key for each word and value of 0.
    full_vocab_dict = {i:0 for i in vocab_set}
    
    # Loop through each word in full_vocab_dict
    for word, val in full_vocab_dict.items():
        docs = 0
        
        # Loop through list of dicts.  Each time a dictionary contains the word, increment docs by 1
        for d in list_of_dicts:
            if word in d:
                docs += 1
        
        # Now that we know denominator for equation, compute and set IDF value for word
        
        full_vocab_dict[word] = np.log((len(list_of_dicts)/ float(docs)))
    
    return full_vocab_dict

In [ ]:
inverse_document_frequency(list_of_dictionaries)

## Let's Start Modeling!!

### Importing sklean multi-class models

In [3]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report


In [4]:
#define features and target variables

features = df1['lyrics']
target = df1['genre']

In [5]:
#train test split dataset 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=.25, random_state=0)

In [13]:
#instantiate piple model
pipeline_model = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [14]:
#fit training set onto model
pipeline_model.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [15]:
y_pred = pipeline_model.predict(X_test)

In [ ]:
accuracy_score(y_pred, y_test)


In [16]:
f1_score(y_pred, y_test, average='weighted')

0.7118764428275003

#### Pipeline Results - with Stopwords
F1 score of .71 was achieved with stopwords included

### Train Test Split (non-stemmed lyrics)

In [ ]:
#let's use the non-stemmed lyrics as features

features_nonstemmed = df1['lyrics']
target = df1['genre']
#train test split dataset 
X_train, X_test, y_train, y_test = train_test_split(features_nonstemmed, target, test_size=.25, random_state=0)

In [ ]:
#instantiate piple model
pipeline_model = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [ ]:
pipeline_model.fit(X_train, y_train)

In [ ]:
y_pred1 = pipeline_model.predict(X_test)
accuracy_score(y_pred1, y_test)


In [ ]:
y

### Support Vector Machine

In [6]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [8]:
y_pred2 = sgd.predict(X_test)
accuracy_score(y_pred2, y_test)

0.6357150418122155

In [9]:
f1_score(y_pred2, y_test, average='weighted')

/Users/flatironschool/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.748954098857008

#### SVG Results - Stopwords included
An F1 Score of .74 was achived from running an SVG model. Lyrics included stopwords and were not stemmed or lemmatized

### Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=-1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

/Users/flatironschool/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/flatironschool/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/flatironschool/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/Users/flatironschool/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=100000.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [11]:
y_pred3 = logreg.predict(X_test)
accuracy_score(y_pred3, y_test)


0.6923467767545253

In [12]:
f1_score(y_pred3, y_test, average='weighted')

0.6992354442433582

#### Logistic Results - with Stopwords
An f1 score of .69 was achieved from this model. Lyrics included stopwords.

### Classification with Word Embeddings

In [ ]:
# pip install gensim

In [ ]:
from gensim.models import Word2Vec